In [ ]:
import json
import time
import datetime
import requests
import pandas as pd
import os

# path to credentials.json
credentials_path = "../config/credentials.json"

# load API key
with open(credentials_path, 'r') as file:
    credentials = json.load(file)
    api_key = credentials.get("riot_api_key")

In [3]:
import requests
import pandas as pd
import os

# API headers
headers = {
    'X-Riot-Token': api_key
}

# base API endpoint
riot_base_url = "https://<region>.api.riotgames.com/lol"

In [13]:
'''
Functions to grab champion and item data
'''

def fetch_champion_data_ddragon():
    """Fetch champion data from DDragon."""
    url = "https://ddragon.leagueoflegends.com/cdn/14.23.1/data/en_US/champion.json"  # Replace with the latest patch
    response = requests.get(url)
    
    if response.status_code == 200:
        champions = response.json()
        return champions
    else:
        print(f"Error fetching champion data: {response.status_code}")
        return None

def fetch_item_data_ddragon():
    """Fetch item data from DDragon."""
    url = "http://ddragon.leagueoflegends.com/cdn/14.23.1/data/en_US/item.json"  # Replace with the latest patch
    response = requests.get(url)
    
    if response.status_code == 200:
        champions = response.json()
        return champions
    else:
        print(f"Error fetching champion data: {response.status_code}")
        return None

In [18]:
# Fetch champion data using DDragon
champion_data = fetch_champion_data_ddragon()

# Fetch item data using DDragon
item_data = fetch_item_data_ddragon()

# Save to JSON file
if champion_data:
    output_path = "../data/raw/champion_data/champions.json"
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "w") as file:
        json.dump(champion_data, file)
    print(f"Champion data saved to {output_path}")

if item_data:
    output_path = "../data/raw/item_data/items.json"
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, "w") as file:
        json.dump(item_data, file)
    print(f"Champion data saved to {output_path}")

Champion data saved to ../data/raw/champion_data/champions.json
Champion data saved to ../data/raw/item_data/items.json


In [71]:
import os
import json
import time
import requests
import datetime

def fetch_summoner_ids_by_multiple_ranks(api_key, region, output_path):
    """
    Fetch summoner IDs by multiple ranks and divisions, and append to a JSON file.

    Parameters:
        api_key (str): Riot API key.
        region (str): Riot region for league data (e.g., "na1").
        output_path (str): Path to save the output JSON file.

    Returns:
        dict: Updated dictionary of summoner IDs categorized by rank.
    """
    # Define the tiers and divisions to iterate over
    tiers = {
        "DIAMOND": ["IV", "III", "II", "I"]
    }

    high_tiers = ["MASTER", "GRANDMASTER", "CHALLENGER"]

    # Initialize dictionary to hold all summoner IDs
    all_summoner_ids = {"DIAMOND": [], "MASTER": [], "GRANDMASTER": [], "CHALLENGER": []}

    # Initialize rate limit tracking
    request_count = 0
    start_time = datetime.datetime.now()

    try:
        # Load existing data if file exists, ensuring correct format
        if os.path.exists(output_path):
            with open(output_path, 'r') as f:
                existing_summoner_ids = json.load(f)
                if not isinstance(existing_summoner_ids, dict):
                    # Reformat existing data to dictionary if it's a list or incorrect format
                    existing_summoner_ids = {"DIAMOND": [], "MASTER": [], "GRANDMASTER": [], "CHALLENGER": []}
        else:
            existing_summoner_ids = {"DIAMOND": [], "MASTER": [], "GRANDMASTER": [], "CHALLENGER": []}

        # Fetch Diamond Summoner IDs
        for tier, divisions in tiers.items():
            for division in divisions:
                page = 1
                while True:
                    # Check rate limit: 20 requests per second, 100 requests per 2 minutes
                    current_time = datetime.datetime.now()
                    elapsed_seconds = (current_time - start_time).total_seconds()

                    # If 2 minutes have passed, reset the request count and start time
                    if elapsed_seconds > 120:
                        request_count = 0
                        start_time = current_time

                    # If we're close to the 100 requests per 2-minute limit, wait
                    if request_count >= 90:
                        sleep_time = 120 - elapsed_seconds
                        print(f"Approaching 2-minute rate limit. Waiting for {sleep_time:.2f} seconds...")
                        time.sleep(sleep_time)
                        request_count = 0
                        start_time = datetime.datetime.now()

                    # Build the URL for Diamond tier
                    url = f"https://{region}.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{tier}/{division}"
                    params = {"page": page}
                    headers = {"X-Riot-Token": api_key}

                    try:
                        # Make the request
                        response = requests.get(url, headers=headers, params=params)
                        request_count += 1

                        # Check for Too Many Requests (Rate Limit)
                        if response.status_code == 429:
                            retry_after = int(response.headers.get("Retry-After", 10))  # Use Retry-After if provided, otherwise 10 seconds
                            print(f"Rate limit exceeded, waiting for {retry_after} seconds...")
                            time.sleep(retry_after)
                            continue

                        # Raise other HTTP errors
                        response.raise_for_status()

                        # Parse the response and extract summoner IDs
                        summoner_entries = response.json()

                        # Stop if no more data is available on this page
                        if not summoner_entries:
                            break

                        # Extract summoner IDs
                        new_summoner_ids = [entry["summonerId"] for entry in summoner_entries]
                        all_summoner_ids["DIAMOND"].extend(new_summoner_ids)
                        page += 1  # Move to the next page

                        # Add a delay to avoid exceeding 20 requests per second
                        if request_count % 20 == 0:
                            print("20 requests made in the last second. Pausing for 1 second to avoid limit...")
                            time.sleep(1)

                    except requests.exceptions.RequestException as e:
                        print(f"Request error for {tier} {division}, page {page}: {e}")
                        break

        # Fetch High Tier Summoner IDs (Master, Grandmaster, Challenger)
        for high_tier in high_tiers:
            # Use the specific endpoint for each high tier
            url = f"https://{region}.api.riotgames.com/lol/league/v4/{high_tier.lower()}leagues/by-queue/RANKED_SOLO_5x5"
            headers = {"X-Riot-Token": api_key}

            try:
                # Make the request
                response = requests.get(url, headers=headers)
                request_count += 1

                # Check for Too Many Requests (Rate Limit)
                if response.status_code == 429:
                    retry_after = int(response.headers.get("Retry-After", 10))  # Use Retry-After if provided, otherwise 10 seconds
                    print(f"Rate limit exceeded, waiting for {retry_after} seconds...")
                    time.sleep(retry_after)
                    continue

                # Raise other HTTP errors
                response.raise_for_status()

                # Parse the response and extract summoner IDs
                summoner_entries = response.json()["entries"]
                new_summoner_ids = [entry["summonerId"] for entry in summoner_entries]
                all_summoner_ids[high_tier].extend(new_summoner_ids)

                # Add a delay to avoid exceeding 20 requests per second
                if request_count % 20 == 0:
                    print("20 requests made in the last second. Pausing for 1 second to avoid limit...")
                    time.sleep(1)

            except requests.exceptions.RequestException as e:
                print(f"Request error for {high_tier}: {e}")
                continue

        # Merge and remove duplicates for each rank
        for rank in all_summoner_ids:
            all_summoner_ids[rank] = list(set(existing_summoner_ids.get(rank, []) + all_summoner_ids[rank]))

        # Ensure output directory exists
        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        # Write updated data to file
        with open(output_path, 'w') as f:
            json.dump(all_summoner_ids, f, indent=4)

        print(f"Summoner IDs updated and saved to {output_path}")
        return all_summoner_ids

    except Exception as e:
        print(f"Unexpected error: {e}")
        return {}

# Example usage
API_KEY = api_key
REGION = "na1"
OUTPUT_PATH = "../data/raw/summoner_id/summoner_id.json"

summoner_ids = fetch_summoner_ids_by_multiple_ranks(
    api_key=API_KEY, 
    region=REGION, 
    output_path=OUTPUT_PATH
)

20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
Approaching 2-minute rate limit. Waiting for 82.10 seconds...
20 requests made in the last second. Pausing for 1 second to avoid limit...
Summoner IDs updated and saved to ../data/raw/summoner_id/summoner_id.json


In [ ]:
def fetch_puuids(api_key, summoner_ids_path, region, output_path):
    """
    Fetch PUUIDs for summoners given their summoner IDs.

    Parameters:
    - api_key: str, your Riot Games API key.
    - summoner_ids_path: str, path to the JSON file containing summoner IDs.
    - region: str, region for summoner data (e.g., "na1").
    - output_path: str, path to save the output JSON file for PUUIDs.

    Returns:
    - puuids: list, list of PUUIDs for all summoners.
    """
    # Load summoner IDs from the given JSON file
    if not os.path.exists(summoner_ids_path):
        print(f"Summoner IDs file not found: {summoner_ids_path}")
        return []

    with open(summoner_ids_path, 'r') as f:
        summoner_data = json.load(f)

    # Combine all summoner IDs from different branches into a single list
    summoner_ids = []
    for rank in summoner_data:
        summoner_ids.extend(summoner_data[rank])

    puuids = []
    headers = {"X-Riot-Token": api_key}

    # Load existing PUUIDs if the output file exists
    if os.path.exists(output_path):
        with open(output_path, 'r') as f:
            existing_puuids = json.load(f)
    else:
        existing_puuids = []

    # Initialize rate limit tracking
    request_count = 0
    start_time = datetime.datetime.now()

    for summoner_id in summoner_ids:
        # Check rate limits
        current_time = datetime.datetime.now()
        elapsed_seconds = (current_time - start_time).total_seconds()

        # Reset the request count after 2 minutes
        if elapsed_seconds > 120:
            request_count = 0
            start_time = current_time

        # If we're approaching the 2-minute limit, wait
        if request_count >= 90:
            sleep_time = 120 - elapsed_seconds
            print(f"Approaching 2-minute rate limit. Waiting for {sleep_time:.2f} seconds...")
            time.sleep(sleep_time)
            request_count = 0
            start_time = datetime.datetime.now()

        # Fetch PUUID using summonerId
        try:
            url = f"https://{region}.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}"
            response = requests.get(url, headers=headers)
            request_count += 1

            # Rate limit per second (20 requests per second)
            if request_count % 20 == 0:
                print("20 requests made in the last second. Pausing for 1 second to avoid limit...")
                time.sleep(1)

            response.raise_for_status()
            summoner_data = response.json()
            puuid = summoner_data["puuid"]
            if puuid not in existing_puuids and puuid not in puuids:
                puuids.append(puuid)
        except requests.exceptions.RequestException as e:
            print(f"Error fetching PUUID for summonerId {summoner_id}: {e}")
            continue

    # Combine new PUUIDs with existing ones and remove duplicates
    all_puuids = list(set(existing_puuids + puuids))

    # Ensure the output directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Save updated PUUIDs to JSON file
    with open(output_path, 'w') as f:
        json.dump(all_puuids, f, indent=4)

    print(f"PUUIDs updated and saved to {output_path}")
    return all_puuids

# Example Usage
API_KEY = api_key
SUMMONER_IDS_PATH = "../data/raw/summoner_id/summoner_id.json"
REGION = "na1"
OUTPUT_PATH = "../data/raw/puuids/puuids.json"

puuids = fetch_puuids(
    api_key=API_KEY, 
    summoner_ids_path=SUMMONER_IDS_PATH, 
    region=REGION, 
    output_path=OUTPUT_PATH
)

20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
Approaching 2-minute rate limit. Waiting for 92.47 seconds...
20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
Approaching 2-minute rate limit. Waiting for 92.41 seconds...
20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pausing for 1 second to avoid limit...
20 requests made in the last second. Pau

KeyboardInterrupt: 

In [ ]:
'''

WHEN WORKING ON THIS TOMORROW, PUT SUMMONERID FROM INITIAL FUNCTION INTO LISTS, AND INPUT THAT LIST INTO SECOND FUNCTION, MAKE SECOND FUNCTION TARGET ONLY RANKED MATCHES

ALSO MAKE SURE SECOND FUNCTION CHECKS IF ID EXISTS WITHIN JSON FILE ALREADY, IF IT DOES DO NOT GRAB AND GET NEW ID



'''

In [22]:
'''
Fetching Complete Match Data
'''

def fetch_match_details(api_key, match_id, region, item_data, champion_data):
    """
    Fetch match details and enrich them with item and champion data.

    Parameters:
    - api_key: str, your Riot Games API key.
    - match_id: str, match ID.
    - region: str, region for match data (e.g., "americas").
    - item_data: dict, item data fetched from Data Dragon.
    - champion_data: dict, champion data fetched from Data Dragon.

    Returns:
    - match_summary: dict, structured data about the match.
    """
    url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match_id}"
    headers = {"X-Riot-Token": api_key}

    response = requests.get(url, headers=headers)
    response.raise_for_status()
    match_data = response.json()

    teams = match_data["info"]["teams"]
    participants = match_data["info"]["participants"]

    # Determine the winning team
    winning_team_id = next(team["teamId"] for team in teams if team["win"])
    winning_team = "Blue" if winning_team_id == 100 else "Red"

    # Extract details for each player
    players = []
    for participant in participants:
        player_data = {
            "summonerName": participant["summonerName"],
            "champion": champion_data.get(participant["championId"], "Unknown Champion"),
            "kills": participant["kills"],
            "deaths": participant["deaths"],
            "assists": participant["assists"],
            "items": [
                item_data.get(str(participant[f"item{i}"]), {}).get("name", "Unknown Item")
                for i in range(6)
            ],
        }
        players.append(player_data)

    match_summary = {
        "winning_team": winning_team,
        "players": players,
    }
    return match_summary

region = 'na1'
match_id = 'NA1_5161885170'

fetch_match_details(api_key, match_id, region, item_data, champion_data)

HTTPError: 400 Client Error: Bad Request for url: https://na1.api.riotgames.com/lol/match/v5/matches/NA1_5161885170

In [32]:
def fetch_latest_patch():
    """
    Fetch the latest game patch version from Riot's Data Dragon.

    Returns:
    - patch: str, the latest patch version.
    """
    url = "https://ddragon.leagueoflegends.com/api/versions.json"
    response = requests.get(url)
    response.raise_for_status()
    patches = response.json()
    return patches[0]  # The latest version is the first in the list

def fetch_item_data(patch):
    """
    Fetch item data from Riot's Data Dragon.

    Parameters:
    - patch: str, the current game version.

    Returns:
    - items: dict, a dictionary mapping item IDs to item details.
    """
    url = f"http://ddragon.leagueoflegends.com/cdn/{patch}/data/en_US/item.json"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    items = data["data"]  # All items are stored under "data"
    return items

def fetch_champion_data(patch):
    """
    Fetch champion data from Riot's Data Dragon.

    Parameters:
    - patch: str, the current game version.

    Returns:
    - champions: dict, a dictionary mapping champion IDs to champion names.
    """
    url = f"http://ddragon.leagueoflegends.com/cdn/{patch}/data/en_US/champion.json"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    champions = {int(champ["key"]): champ["name"] for champ in data["data"].values()}
    return champions

def fetch_match_details(api_key, match_id, region, item_data, champion_data):
    """
    Fetch match details and enrich them with item and champion data.

    Parameters:
    - api_key: str, your Riot Games API key.
    - match_id: str, match ID.
    - region: str, region for match data (e.g., "americas").
    - item_data: dict, item data fetched from Data Dragon.
    - champion_data: dict, champion data fetched from Data Dragon.

    Returns:
    - match_summary: dict, structured data about the match.
    """
    url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match_id}"
    headers = {"X-Riot-Token": api_key}

    response = requests.get(url, headers=headers)
    response.raise_for_status()
    match_data = response.json()

    teams = match_data["info"]["teams"]
    participants = match_data["info"]["participants"]

    # Determine the winning team
    winning_team_id = next(team["teamId"] for team in teams if team["win"])
    winning_team = "Blue" if winning_team_id == 100 else "Red"

    # Extract details for each player
    players = []
    for participant in participants:
        player_data = {
            "summonerName": participant["summonerName"],
            "champion": champion_data.get(participant["championId"], "Unknown Champion"),
            "kills": participant["kills"],
            "deaths": participant["deaths"],
            "assists": participant["assists"],
            "items": [
                item_data.get(str(participant[f"item{i}"]), {}).get("name", "Unknown Item")
                for i in range(6)
            ],
        }
        players.append(player_data)

    match_summary = {
        "winning_team": winning_team,
        "players": players,
    }
    return match_summary

# Example Usage
def main():
    api_key = credentials.get("riot_api_key")
    match_id = "NA1_5161885170"  # Example match ID
    match_region = "americas"

    # Fetch the latest patch, items, and champions
    patch = fetch_latest_patch()
    item_data = fetch_item_data(patch)
    champion_data = fetch_champion_data(patch)

    # Fetch and display match details
    match_summary = fetch_match_details(api_key, match_id, match_region, item_data, champion_data)

    # Print the match summary
    print(f"Winning Team: {match_summary['winning_team']}")
    for player in match_summary["players"]:
        print(
            f"{player['summonerName']} played {player['champion']} with "
            f"{player['kills']}/{player['deaths']}/{player['assists']} and built "
            f"{', '.join(player['items'])}"
        )

if __name__ == "__main__":
    main()

Winning Team: Red
Frobenius played Tahm Kench with 8/13/14 and built Heartsteel, Thornmail, Sunfire Aegis, Unknown Item, Unknown Item, Mercury's Treads
EreshKidu played Seraphine with 2/16/19 and built Malignance, Refillable Potion, Ionian Boots of Lucidity, Oblivion Orb, Liandry's Torment, Blasting Wand
Neo Exile played Mordekaiser with 10/16/18 and built Liandry's Torment, Plated Steelcaps, Rylai's Crystal Scepter, Thornmail, Chain Vest, Bami's Cinder
Opalight played Xayah with 7/14/16 and built Kraken Slayer, Navori Flickerblade, Mortal Reminder, Berserker's Greaves, Serrated Dirk, Unknown Item
ERNXST played Ezreal with 11/12/15 and built Chempunk Chainsword, Pickaxe, Long Sword, Muramana, Ionian Boots of Lucidity, Trinity Force
ICanHazWaffles played Taliyah with 7/6/30 and built Guardian's Orb, Tear of the Goddess, Liandry's Torment, Sorcerer's Shoes, Blackfire Torch, Blasting Wand
Gemici played Vel'Koz with 19/8/26 and built Malignance, Refillable Potion, Sorcerer's Shoes, Shadowf